In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
app_data = pd.read_csv("/Users/mayankverma/CMPE-255/Group_Project/Google-Playstore.csv")
app_data.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


In [3]:
app_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), float64(4), int64(1), object(15)
memor

In [4]:
app_data.shape

(2312944, 24)

In [5]:
#  'Size', 'Minimum Android', retained

In [6]:
app_data.drop(['App Id', 'Currency', 'Developer Id', 'Developer Website', 'Developer Email', 'Privacy Policy', 'Editors Choice', 'Scraped Time'], axis=1, inplace=True)
app_data.head()

,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Size,Minimum Android,Released,Last Updated,Content Rating,Ad Supported,In App Purchases
0,Gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,10M,7.1 and up,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False
1,Ampere Battery Info,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,2.9M,5.0 and up,"May 21, 2020","May 06, 2021",Everyone,True,False
2,Vibook,Productivity,0.0,0.0,50+,50.0,58,True,0.0,3.7M,4.0.3 and up,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,10+,10.0,19,True,0.0,1.8M,4.0.3 and up,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False
4,GROW.me,Tools,0.0,0.0,100+,100.0,478,True,0.0,6.2M,4.1 and up,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False


In [7]:
print(app_data.isnull().sum())

App Name                2
Category                0
Rating              22883
Rating Count        22883
Installs              107
Minimum Installs      107
Maximum Installs        0
Free                    0
Price                   0
Size                  196
Minimum Android      6530
Released            71053
Last Updated            0
Content Rating          0
Ad Supported            0
In App Purchases        0
dtype: int64


In [8]:
app_data.shape

(2312944, 16)

In [9]:
app_data.dropna(subset=['Rating', 'Rating Count', 'Installs', 'Size', 'Minimum Android'], inplace=True)
app_data.shape

(2283535, 16)

Preprocessing Content Rating
Mature 17+      => Mature
Adults only 18+ => Adults
Everyone 10+    => Everyone

In [10]:
app_data['Content Rating'].value_counts()

Everyone           1996767
Teen                193733
Mature 17+           59590
Everyone 10+         33160
Unrated                154
Adults only 18+        131
Name: Content Rating, dtype: int64

In [11]:
app_data['Content Rating'] = app_data['Content Rating'].replace('Unrated',"Everyone")

#Cleaning other values just to include Everyone, Teens and Adult 

app_data['Content Rating'] = app_data['Content Rating'].replace('Mature 17+',"Mature")
app_data['Content Rating'] = app_data['Content Rating'].replace('Adults only 18+',"Adults")
app_data['Content Rating'] = app_data['Content Rating'].replace('Everyone 10+',"Everyone")

Preprocessing Content Installs
replaced commmas with no space & dropped + s

In [12]:
app_data.Installs = app_data.Installs.str.replace(',','')
app_data.Installs = app_data.Installs.str.replace('+','')

<ipython-input-12-e24377e720dd>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  app_data.Installs = app_data.Installs.str.replace('+','')


In [13]:
app_data['Installs'] = pd.to_numeric(app_data['Installs'])
app_data['Installs'] = app_data.apply( lambda row: math.log2(row.Installs+1), axis = 1)

In [14]:
app_data.head()

,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Size,Minimum Android,Released,Last Updated,Content Rating,Ad Supported,In App Purchases
0,Gakondo,Adventure,0.0,0.0,3.459432,10.0,15,True,0.0,10M,7.1 and up,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False
1,Ampere Battery Info,Tools,4.4,64.0,12.288001,5000.0,7662,True,0.0,2.9M,5.0 and up,"May 21, 2020","May 06, 2021",Everyone,True,False
2,Vibook,Productivity,0.0,0.0,5.672425,50.0,58,True,0.0,3.7M,4.0.3 and up,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,3.459432,10.0,19,True,0.0,1.8M,4.0.3 and up,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False
4,GROW.me,Tools,0.0,0.0,6.658211,100.0,478,True,0.0,6.2M,4.1 and up,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False


Converting app size from string to numeric

In [15]:
def get_app_size( size_str):
    if size_str == 'Varies with device':
        return 0
    metric_unit = size_str[-1]
    val = float(size_str[:-1])
    if metric_unit == 'k' or metric_unit == 'K':
        val = val*1000.0
    elif metric_unit == 'M':
        val = val*1000000.0
    elif metric_unit == 'G':
        val = val*1000000000.0
    else:
        raise Exception(" Value metric not detected {} ".format(size_str))
    return math.log2(val)

In [16]:
app_data.Size = app_data.Size.str.replace(',','')
app_data['Size_Log'] = app_data.apply( lambda row: get_app_size(row.Size), axis = 1)

In [17]:
def get_version_value( version):
    if version == 'up':
        return 11
    if version[-1] == 'W':
        version = version[:-1]
    version = version.split('.')
    version.append('0')
    return float("{}.{}".format(version[0], version[1]))

In [18]:
def get_min_version( version):
    if version == 'Varies with device':
        return 0
    elif '-' in version:
        arr = version.split('-')
        return get_version_value(arr[0].strip())
    elif 'and' in version:
        arr = version.split('and')
        return get_version_value(arr[0].strip())
    return get_version_value( version)

In [19]:
def get_max_version( version):
    if version == 'Varies with device':
        return 11
    elif '-' in version:
        arr = version.split('-')
        return get_version_value(arr[1].strip())
    elif 'and' in version:
        arr = version.split('and')
        return get_version_value(arr[1].strip())
    return get_version_value( version)

In [20]:
app_data['Min_Version'] = app_data.apply( lambda row: get_min_version(row['Minimum Android']), axis = 1)
app_data['Max_Version'] = app_data.apply( lambda row: get_max_version(row['Minimum Android']), axis = 1)

In [21]:
app_data['Category'] = pd.factorize(app_data['Category'])[0].astype(int)
app_data['Content Rating'] = pd.factorize(app_data['Content Rating'])[0].astype(int)
app_data['Ad Supported'] = pd.factorize(app_data['Ad Supported'])[0].astype(int)
app_data['In App Purchases'] = pd.factorize(app_data['In App Purchases'])[0].astype(int)
app_data['Free'] = pd.factorize(app_data['Free'])[0].astype(int)
app_data.head()

,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Size,Minimum Android,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Size_Log,Min_Version,Max_Version
0,Gakondo,0,0.0,0.0,3.459432,10.0,15,0,0.0,10M,7.1 and up,"Feb 26, 2020","Feb 26, 2020",0,0,0,23.253497,7.1,11.0
1,Ampere Battery Info,1,4.4,64.0,12.288001,5000.0,7662,0,0.0,2.9M,5.0 and up,"May 21, 2020","May 06, 2021",0,1,0,21.467621,5.0,11.0
2,Vibook,2,0.0,0.0,5.672425,50.0,58,0,0.0,3.7M,4.0.3 and up,"Aug 9, 2019","Aug 19, 2019",0,0,0,21.819094,4.0,11.0
3,Smart City Trichy Public Service Vehicles 17UC...,3,5.0,5.0,3.459432,10.0,19,0,0.0,1.8M,4.0.3 and up,"Sep 10, 2018","Oct 13, 2018",0,1,0,20.779565,4.0,11.0
4,GROW.me,1,0.0,0.0,6.658211,100.0,478,0,0.0,6.2M,4.1 and up,"Feb 21, 2020","Nov 12, 2018",0,0,0,22.563837,4.1,11.0


In [22]:
app_data['Content Rating'].value_counts()

0    2030081
1     193733
2      59590
3        131
Name: Content Rating, dtype: int64

In [23]:
X = app_data.drop(['App Name', 'Size', 'Content Rating', 'Minimum Android','Rating', 'Rating Count', 'Minimum Installs', 'Maximum Installs', 'Price', 'Released', 'Last Updated', 'Free'], axis=1)
X.head()

,Category,Installs,Ad Supported,In App Purchases,Size_Log,Min_Version,Max_Version
0,0,3.459432,0,0,23.253497,7.1,11.0
1,1,12.288001,1,0,21.467621,5.0,11.0
2,2,5.672425,0,0,21.819094,4.0,11.0
3,3,3.459432,1,0,20.779565,4.0,11.0
4,1,6.658211,0,0,22.563837,4.1,11.0


In [24]:
y = app_data['Rating'].values.round()
print(y[0:4])

[0. 4. 0. 5.]


In [25]:
std_scaler = StandardScaler()
X['Installs'] = std_scaler.fit_transform(X[['Installs']])
X['Size_Log'] = std_scaler.fit_transform(X[['Size_Log']])
X['Min_Version'] = std_scaler.fit_transform(X[['Min_Version']])
X['Max_Version'] = std_scaler.fit_transform(X[['Max_Version']])
X.head()

,Category,Installs,Ad Supported,In App Purchases,Size_Log,Min_Version,Max_Version
0,0,-1.207233,0,0,0.115326,2.892475,0.013758
1,1,0.741734,1,0,-0.346256,0.720834,0.013758
2,2,-0.718699,0,0,-0.255413,-0.313281,0.013758
3,3,-1.207233,1,0,-0.524092,-0.313281,0.013758
4,1,-0.501080,0,0,-0.062925,-0.209870,0.013758


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [37]:
count = {}
for i in y_train:
    if i not in count:
        count[i] = 0
    count[i] = count[i] + 1
print(count)

{5.0: 276645, 4.0: 552129, 0.0: 844407, 3.0: 118383, 2.0: 32831, 1.0: 2433}


In [168]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
rf_acc = accuracy_score(y_pred,y_test)*100
print("Accuracy =",round(rf_acc,2),"%")

Accuracy = 65.12 %


In [176]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit( X_train, y_train)
y_pred = clf.predict(X_test)
rf_acc = accuracy_score(y_pred, y_test)*100
print("Accuracy =",round(rf_acc,2),"%")

Accuracy = 68.97 %


/Users/mayankverma/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
clf = svm.SVC()
# clf = GaussianNB()
# clf = MultinomialNB()
# clf = BernoulliNB()
clf.fit( X_train, y_train)
y_pred = clf.predict(X_test)
rf_acc = accuracy_score(y_pred, y_test)*100
print("Accuracy =",round(rf_acc,2),"%")

In [27]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
clf.fit( X_train, y_train)
y_pred = clf.predict(X_test)
rf_acc = accuracy_score(y_pred, y_test)*100
print("Accuracy =",round(rf_acc,2),"%")

Accuracy = 68.99 %


In [33]:
confusion_matrix(y_pred, y_test)

array([[193136,    182,   1050,   3337,  15964,  31453],
       [     0,      0,      0,      0,      0,      0],
       [     0,      0,      0,      0,      0,      0],
       [     1,      0,      0,      0,      2,      0],
       [ 17909,    440,   7120,  26281, 121939,  37450],
       [   116,      2,      5,     13,    123,    184]])

In [120]:
res[-1]

'M'

In [121]:
res[:-1]

'551'

In [152]:
res = ''
for val in set(app_data['Minimum Android']):
    res = res + val + '    ==>    ' + str(get_min_version(val)) + ' to ' + str(get_max_version(val)) + '\n'
with open('temp.txt', 'w') as fw:
    fw.write(res)